In [20]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

In [21]:
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, END
from langchain_core.messages import HumanMessage, SystemMessage
from typing import TypedDict

llm = ChatOpenAI(
    api_key=os.getenv("OPENAI_API_KEY"),
    model="gpt-3.5-turbo"
)

system_message = SystemMessage(content="Eres una calculadora diseñada para resolver las operaciones básicas")


class State(TypedDict):
  first_number: int
  second_number: int
  result: int

In [22]:
GET_FIRST_NUMBER_NODE = "get_first_number_node"
GET_SECOND_NUMBER_NODE = "get_second_number_node"
GET_RESULT_NODE = "get_result_node"

In [33]:
def get_first_number_node(state):
  human_message = HumanMessage(
      content="Genera un número aleatorio del 1 al 10 y devuélvelo solo como un número, sin texto adicional.")
  messages = [system_message, human_message]
  ia_response = llm.invoke(messages)
  response = int(ia_response.content)
  state["first_number"] = response
  print(f"Primer número: {state["first_number"]}")
  return state

def get_second_number_node(state):
  human_message = HumanMessage(
      content="Genera un número aleatorio del 1 al 10 y devuélvelo solo como un número, sin texto adicional.")
  messages = [system_message, human_message]
  ia_response = llm.invoke(messages)
  response = int(ia_response.content)
  state["second_number"] = response
  print(f"Segundo número: {state["second_number"]}")
  return state

def get_result_node(state):
  first_number = state["first_number"]
  second_number = state["second_number"]

  human_message = HumanMessage(content=f"Suma {first_number} y {second_number}, finalmente devuélvelo solo como un número, sin texto adicional.")
  messages = [system_message, human_message]
  ia_response = llm.invoke(messages)
  response = int(ia_response.content)
  state["result"] = response
  print(f"El resultado de la suma entre {first_number} y {second_number} es: {response}")
  return state

In [34]:
workflow = StateGraph(State)

workflow.add_node(GET_FIRST_NUMBER_NODE, get_first_number_node)
workflow.add_node(GET_SECOND_NUMBER_NODE, get_second_number_node)
workflow.add_node(GET_RESULT_NODE, get_result_node)

workflow.set_entry_point(GET_FIRST_NUMBER_NODE)
workflow.set_finish_point(GET_RESULT_NODE)

workflow.add_edge(GET_FIRST_NUMBER_NODE, GET_SECOND_NUMBER_NODE)
workflow.add_edge(GET_SECOND_NUMBER_NODE, GET_RESULT_NODE)


In [35]:
agent = workflow.compile()

In [36]:
initial_state = {"first_number": 0, "second_number": 0, "result": 0}
agent.invoke(initial_state)

Primer número: 8
Segundo número: 7
El resultado de la suma entre 8 y 7 es: 15


{'first_number': 8, 'second_number': 7, 'result': 15}

In [37]:
print(agent.get_graph().draw_ascii())

      +-----------+        
      | __start__ |        
      +-----------+        
             *             
             *             
             *             
+-----------------------+  
| get_first_number_node |  
+-----------------------+  
             *             
             *             
             *             
+------------------------+ 
| get_second_number_node | 
+------------------------+ 
             *             
             *             
             *             
    +-----------------+    
    | get_result_node |    
    +-----------------+    
             *             
             *             
             *             
        +---------+        
        | __end__ |        
        +---------+        
